In [1]:
import pandas as pd
import scipy.io
from deepface import DeepFace
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import scipy
import pandas as pd
import numpy as np
import cv2
import time
import face_recognition
import tarfile
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import gc
from joblib import Parallel, delayed
import pandas as pd
import scipy.io
from deepface import DeepFace
import logging
logging.getLogger("deepface").setLevel(logging.ERROR)

2024-04-19 19:52:08.971674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Load Data from the .mat into a pandas dataframe
mat = scipy.io.loadmat('imdb_crop/imdb.mat')
columns = ["dob", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score",
           "celeb_names", "celeb_id"]
instances = mat['imdb'][0][0][0].shape[1]
df = pd.DataFrame(index=range(0, instances), columns=columns)

for i in mat:
    if i == "imdb":
        current_array = mat[i][0][0]
        for j in range(len(current_array)):
            df[columns[j]] = pd.DataFrame(current_array[j][0])

# remove pictures that don't have face
df = df[df['face_score'] != -np.inf]

# remove pictures with more than one face
df = df[df['second_face_score'].isna()]

# only take high quality photos
df = df[df['face_score'] >= 3]
 

In [59]:
#df_og = df 
#df = df.iloc[0:10]

In [ ]:
def getImagePixels(image_path):
   return cv2.imread("imdb_crop/" + image_path[0])

# Applying parallel image loading to help speed up the code
def parallel_get_image(image_paths):
    # To add a progress bar, we wrap image_paths with `tqdm`
    with ThreadPoolExecutor(max_workers=10) as executor:
        images = list(tqdm(executor.map(getImagePixels, image_paths), total=len(image_paths), desc="Loading Images"))
    return images


df['pixels'] = parallel_get_image(df['full_path'])
 
#df['pixels'] = df['full_path'].apply(getImagePixels)

In [6]:
import pandas as pd
from deepface import DeepFace
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def findFaceRepresentation(img):
    try:
        # Assuming 'img' is a path or a valid image format for DeepFace
        representation = DeepFace.represent(img_path=img, model_name="VGG-Face", detector_backend="skip")
    except Exception as e:
        print(f"Error processing image {img}: {e}")
        representation = None
    return representation

def apply_parallel(df, func):
    # Number of workers (threads); adjust based on your system capabilities
    num_workers = 10
    # Create a thread pool for parallel processing
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Set up a progress bar
        tasks = [executor.submit(func, img) for img in df['pixels']]
        results = []
        for future in tqdm(as_completed(tasks), total=len(tasks), desc="Processing images"):
            results.append(future.result())
        return results

# Assuming 'df' is your DataFrame and 'pixels' contains image paths or data
df['face_vector_raw'] = apply_parallel(df, findFaceRepresentation)


Processing images: 100%|██████████| 95234/95234 [6:26:59<00:00,  4.10it/s]      


# Save as pickle!

Uncomment the line below

In [7]:
#df.to_pickle("./df_with_embeddings.pkl")

Then, to load the pickle when you want to use it, use 

`unpickled_df = pd.read_pickle("./df_with_embeddings.pkl")`